In [ ]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.ethics.ex4 import *
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data, separate features from target
data = pd.read_csv("../input/synthetic-credit-card-approval/synthetic_credit_card_approval.csv")
X = data.drop(["Target"], axis=1)
y = data["Target"]

# Break into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Preview the data
print("Data successfully loaded!\n")
X_train.head()

The dataset contains, for each applicant:

* income (in the Income column),
* the number of children (in the Num_Children column),
* whether the applicant owns a car (in the Own_Car column, the value is 1 if the applicant owns a car, and is else 0), and
* whether the applicant owns a home (in the Own_Housing column, the value is 1 if the applicant owns a home, and is else 0)

When evaluating fairness, we'll check how the model performs for users in different groups, as identified by the Group column:

* The Group column breaks the users into two groups (where each group corresponds to either 0 or 1).
* For instance, you can think of the column as breaking the users into two different races, ethnicities, or gender groupings. If the column breaks users into different ethnicities, 0 could correspond to a non-Hispanic user, while 1 corresponds to a Hispanic user.

In [ ]:
from sklearn import tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Train a model and make predictions
model_baseline = tree.DecisionTreeClassifier(random_state=0, max_depth=3)
model_baseline.fit(X_train, y_train)
preds_baseline = model_baseline.predict(X_test)

This section of code creates a model and trains it with the Decision Tree Classifier, making predictions based on that ML algorithm. 

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(estimator, X, y_true, y_pred, display_labels=["Deny", "Approve"],
                          include_values=True, xticks_rotation='horizontal', values_format='',
                          normalize=None, cmap=plt.cm.Blues):
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
    return cm, disp.plot(include_values=include_values, cmap=cmap, xticks_rotation=xticks_rotation,
                     values_format=values_format)

# Function to evaluate the fairness of the model
def get_stats(X, y, model, group_one, preds):
        
    y_zero, preds_zero, X_zero = y[group_one==False], preds[group_one==False], X[group_one==False]
    y_one, preds_one, X_one = y[group_one], preds[group_one], X[group_one]
    
    print("Total approvals:", preds.sum())
    print("Group A:", preds_zero.sum(), "({}% of approvals)".format(round(preds_zero.sum()/sum(preds)*100, 2)))
    print("Group B:", preds_one.sum(), "({}% of approvals)".format(round(preds_one.sum()/sum(preds)*100, 2)))
    
    print("\nOverall accuracy: {}%".format(round((preds==y).sum()/len(y)*100, 2)))
    print("Group A: {}%".format(round((preds_zero==y_zero).sum()/len(y_zero)*100, 2)))
    print("Group B: {}%".format(round((preds_one==y_one).sum()/len(y_one)*100, 2)))
    
    cm_zero, disp_zero = plot_confusion_matrix(model, X_zero, y_zero, preds_zero)
    disp_zero.ax_.set_title("Group A")
    cm_one, disp_one = plot_confusion_matrix(model, X_one, y_one, preds_one)
    disp_one.ax_.set_title("Group B")
    
    print("\nSensitivity / True positive rate:")
    print("Group A: {}%".format(round(cm_zero[1,1] / cm_zero[1].sum()*100, 2)))
    print("Group B: {}%".format(round(cm_one[1,1] / cm_one[1].sum()*100, 2)))
    
# Evaluate the model    
get_stats(X_test, y_test, model_baseline, X_test["Group"]==1, preds_baseline)

The confusion matrices above show how the model performs on some test data. We also print additional information (calculated from the confusion matrices) to assess fairness of the model. For instance,

* The model approved 38246 people for a credit card. Of these individuals, 8028 belonged to Group A, and 30218 belonged to Group B.
* The model is 94.56% accurate for Group A, and 95.02% accurate for Group B. These percentages can be calculated directly from the confusion matrix; for instance, for Group A, the accuracy is (39723+7528)/(39723+500+2219+7528).
* The true positive rate (TPR) for Group A is 77.23%, and the TPR for Group B is 98.03%. These percentages can be calculated directly from the confusion matrix; for instance, for Group A, the TPR is 7528/(7528+2219).

# Data Exploration

In [ ]:
#X["Num_Children"].value_counts(normalize=True)
X["Num_Children"].value_counts()

In [ ]:
#X["Own_Car"].value_counts()
X["Own_Car"].value_counts(normalize=True)

In [ ]:
#X["Own_Housing"].value_counts()
X["Own_Housing"].value_counts(normalize=True)

The data from the columns above all show that the current dataset is imbalanced.

In [ ]:
!pip install fairlearn
#install package if missing

In [ ]:
sen_train = X_train["Group"]==1
sen_train.head()

"Sensitive features to identify groups by" -FairLearn

In [ ]:
sen_test = X_test["Group"]==1
sen_test.head()

# PostProcessing

In regards to ML fairness, postprocessing techniques are a series of mitigation algorithms for unfairness that utilize pre-trained models and aim to fit transformation function to that model's outputs in order to improve fairness constraint(s).

# Threshold Optimizer

This algorithm takes previously trained models whose results/predictions act as a score to identify certain thresholds for each group. Then the algorithm optimizes a specified objective metric so that it meets specified fairness constraints.

The code below will use ThresholdOptimizer to maximize the selection_rate compared to the previously trained decision tree in order to make sure the demographic parity is met (selection rate will be the same percentage among groups or at least improved)

**Demographic Parity**

Model is fair if the composition of people who are selected by the model matches the group membership percentages of the applicants

In [ ]:
from fairlearn.postprocessing import ThresholdOptimizer

#model_baseline = tree.DecisionTreeClassifier(random_state=0, max_depth=3)
postprocess_est = ThresholdOptimizer(
                    estimator=model_baseline,
                    constraints='demographic_parity',
                    objective='accuracy_score',
                    prefit=True,
                    predict_method = 'auto')
topt_model = postprocess_est.fit(X_train, y_train, sensitive_features=sen_train)
preds_topt = topt_model.predict(X_test, sensitive_features=sen_test)

In [ ]:
get_stats(X_test, y_test, topt_model, sen_test, preds_topt)

**Before**
> Total approvals: 38246
> > Group A: 8028 (20.99% of approvals)

> > Group B: 30218 (79.01% of approvals)

**After**
> Total approvals: 100000
> > Group A: 49970 (49.97% of approvals)

> > Group B: 50030 (50.03% of approvals)

**Equal Opportunity**

This fairness ensures that the proportion of people who should be selected by the model ("positives") that are correctly selected by the model is the same for each group. This proportion is referred to as the true positive rate (TPR) or sensitivity of the model.

**Equal Accuracy**

The model is fair if the percentage of correct classifications (people who should be denied and are denied, and people who should be approved who are approved) should be the same for each group.

# AIF 360

AI Fairness 360 focuses on removing bias by clarifying and labeling groups as unprivileged and privileged. This is another way for their algorithms to detect protected attributes, which are attributes that partition a population into groups whose outcomes should have parity. Examples include race, gender, caste, and religion. Protected attributes are not universal, but are application specific.

Adult Census Income
Protected Attributes would be:
* -Race, privileged: White, unprivileged: Non-white
* -Sex, privileged: Male, unprivileged: Female

There are 3 postprocessing algorithms offered by the aif360 package
* CalibratedEqOddsPostprocessing
* EqOddsPostprocessing
* RejectObjectClassification

In [ ]:
!pip install aif360
#install package if missing

In [ ]:
priv_group = X_train['Group']==1
unpriv_group = X_train['Group']==0

In [ ]:
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, EqOddsPostprocessing, RejectOptionClassification

In [ ]:
CPP = CalibratedEqOddsPostprocessing(privileged_groups = priv_group,
                                     unprivileged_groups = unpriv_group,
                                     cost_constraint='weighted',
                                     seed=33)

CPP = CPP.fit(model_baseline, preds_baseline)